## Data preprocessing and Model evaluation for 3rd proactive strategy

In [1]:
# Importing necessary libraries for data preprocessing and model evaluation
import os
import gc
import utils
import numpy as np
import pandas as pd
import csv
from tabulate import tabulate
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, mean_absolute_error

#### Loading the models, testing data and predictions

In [2]:
# Loading the models
model_rb = utils.load_and_verify_model("models/rb/model_rb.keras")
print("\n")
model_gl = utils.load_and_verify_model("models/gl/model_gl.keras")
print("\n")
model_tc = utils.load_and_verify_model("models/tc/model_tc.keras")

Model loaded from models/rb/model_rb.keras
The model has an optimizer, indicating it has been trained.
The model has 1660033 parameters.


Model loaded from models/gl/model_gl.keras
The model has an optimizer, indicating it has been trained.
The model has 1660033 parameters.


Model loaded from models/tc/model_tc.keras
The model has an optimizer, indicating it has been trained.
The model has 1660033 parameters.


In [3]:
# Loading the test predictions of MVFF models
predictions_cj = utils.load_data("results/cj/predictions_cj.csv", "csv")
predictions_sj = utils.load_data("results/sj/predictions_sj.csv", "csv")
predictions_tj = utils.load_data("results/tj/predictions_tj.csv", "csv")

In [4]:
# Checking the shape of the predictions
print("Shape of current joints predictions:", predictions_cj.shape)
print("Shape of speed joints predictions:", predictions_sj.shape)
print("Shape of temperature joints predictions:", predictions_tj.shape)

Shape of current joints predictions: (869, 6)
Shape of speed joints predictions: (869, 6)
Shape of temperature joints predictions: (869, 6)


In [5]:
# Loading the testing data
ted_rb = utils.load_data("data/processed/rb/test/scaled_test_data_rb.csv", "csv")
ted_gl = utils.load_data("data/processed/gl/test/scaled_test_data_gl.csv", "csv")
ted_tc = utils.load_data("data/processed/tc/test/scaled_test_data_tc.csv", "csv")

In [6]:
# Loading the test Labels
tel_rb = utils.load_data("data/processed/rb/test/test_labels_rb.csv", "csv")
tel_gl = utils.load_data("data/processed/gl/test/test_labels_gl.csv", "csv")
tel_tc = utils.load_data("data/processed/tc/test/test_labels_tc.csv", "csv")

In [7]:
# Checking the shape of the predictions
print("Shape of current joints data:", ted_rb.shape)
print("Shape of speed joints data:", ted_gl.shape)
print("Shape of temperature joints data:", ted_tc.shape)

Shape of current joints data: (899, 71)
Shape of speed joints data: (899, 71)
Shape of temperature joints data: (899, 71)


#### Data Preprocessing

In [8]:
# Defining the columns to replace test columns
n_pred = len(predictions_cj)
cj_columns = [f'Current_J{i}' for i in range(6)]
sj_columns = [f'Speed_J{i}' for i in range(6)]
tj_columns = ['Temperature_T0'] + [f'Temperature_J{i}' for i in range(1, 6)]

In [9]:
ted_rb.loc[:n_pred-1, cj_columns] = predictions_cj.values
ted_gl.loc[:n_pred-1, cj_columns] = predictions_cj.values
ted_tc.loc[:n_pred-1, cj_columns] = predictions_cj.values

In [10]:
ted_rb.loc[:n_pred-1, sj_columns] = predictions_sj.values
ted_gl.loc[:n_pred-1, sj_columns] = predictions_sj.values
ted_tc.loc[:n_pred-1, sj_columns] = predictions_sj.values

In [11]:
ted_rb.loc[:n_pred-1, tj_columns] = predictions_tj.values
ted_gl.loc[:n_pred-1, tj_columns] = predictions_tj.values
ted_tc.loc[:n_pred-1, tj_columns] = predictions_tj.values

#### Adding interaction features

In [12]:
ted_rb['Average Temperature'] = ted_rb[[f'Temperature_J{i}' for i in range(1, 6)] + ['Temperature_T0']].mean(axis=1)
ted_gl['Average Temperature'] = ted_gl[[f'Temperature_J{i}' for i in range(1, 6)] + ['Temperature_T0']].mean(axis=1)
ted_tc['Average Temperature'] = ted_tc[[f'Temperature_J{i}' for i in range(1, 6)] + ['Temperature_T0']].mean(axis=1)

In [13]:
ted_rb['Gradient Temperature'] = (ted_rb[[f'Temperature_J{i}' for i in range(1, 6)] + ['Temperature_T0']].max(axis=1) - 
                                   ted_rb[[f'Temperature_J{i}' for i in range(1, 6)] + ['Temperature_T0']].min(axis=1))
ted_gl['Gradient Temperature'] = (ted_gl[[f'Temperature_J{i}' for i in range(1, 6)] + ['Temperature_T0']].max(axis=1) - 
                                   ted_gl[[f'Temperature_J{i}' for i in range(1, 6)] + ['Temperature_T0']].min(axis=1))
ted_tc['Gradient Temperature'] = (ted_tc[[f'Temperature_J{i}' for i in range(1, 6)] + ['Temperature_T0']].max(axis=1) - 
                                   ted_tc[[f'Temperature_J{i}' for i in range(1, 6)] + ['Temperature_T0']].min(axis=1))

In [14]:
ted_rb['Load Imbalance'] = (
    ted_rb[[f'Current_J{i}' for i in range(0, 6)]].max(axis=1) - 
    ted_rb[[f'Current_J{i}' for i in range(0, 6)]].min(axis=1)) / ted_rb[[f'Current_J{i}' for i in range(0, 6)]].mean(axis=1)
ted_gl['Load Imbalance'] = (
    ted_gl[[f'Current_J{i}' for i in range(0, 6)]].max(axis=1) - 
    ted_gl[[f'Current_J{i}' for i in range(0, 6)]].min(axis=1)) / ted_gl[[f'Current_J{i}' for i in range(0, 6)]].mean(axis=1)
ted_tc['Load Imbalance'] = (
    ted_tc[[f'Current_J{i}' for i in range(0, 6)]].max(axis=1) - 
    ted_tc[[f'Current_J{i}' for i in range(0, 6)]].min(axis=1)) / ted_tc[[f'Current_J{i}' for i in range(0, 6)]].mean(axis=1)

#### Creating Sequences

In [15]:
# Loading the sequence length value
with open("data/processed/sequence_length.txt", "r") as file:
    sequence_length = int(file.read().strip())

In [16]:
# Creating sequences for test datasets
ted_rb, tel_rb = utils.create_sequences(ted_rb, tel_rb, sequence_length, "classification")
ted_gl, tel_gl = utils.create_sequences(ted_gl, tel_gl, sequence_length, "classification")
ted_tc, tel_tc = utils.create_sequences(ted_tc, tel_tc, sequence_length, "regression")

In [17]:
# Checking if the sequences are created properly
datasets = {
    'Robot Protective Stop': [ted_rb, tel_rb],
    'Grip Lost': [ted_gl, tel_gl],
    'Tool Current': [ted_tc, tel_tc]
}

for label, (x_test, y_test) in datasets.items():
    print(f"Test data \"{label}\": {x_test.shape}, Test labels \"{label}\": {y_test.shape}")

Test data "Robot Protective Stop": (869, 30, 71), Test labels "Robot Protective Stop": (869, 1)
Test data "Grip Lost": (869, 30, 71), Test labels "Grip Lost": (869, 1)
Test data "Tool Current": (869, 30, 71), Test labels "Tool Current": (869, 1)


In [18]:
# Saving the numpy arrays
arrays = {
    "rb": {"test": (ted_rb, tel_rb)},
    "gl": {"test": (ted_gl, tel_gl)},
    "tc": {"test": (ted_tc, tel_tc)}
}

for key, splits in arrays.items():
    for split, data in splits.items():
        folder = f"data/processed/fesf/{key}/{split}/sequences"
        utils.save_sequences(data, folder, f"seq_{split}_data_{key}.npy", f"seq_{split}_labels_{key}.npy")

Saved sequences to: data\processed\fesf\rb\test\sequences
Saved sequences to: data\processed\fesf\gl\test\sequences
Saved sequences to: data\processed\fesf\tc\test\sequences


#### Model Evluation and Results

In [19]:
# Predicting the test data on trained models
probability_rb = model_rb.predict(ted_rb)
probability_gl = model_gl.predict(ted_gl)
predictions_tc = model_tc.predict(ted_tc)

28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 132ms/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step


In [20]:
# Getting the binary predictions of 1st model and 2nd model
predictions_rb = (probability_rb > 0.6).astype(int)
predictions_gl = (probability_gl > 0.4).astype(int)

In [21]:
# Saving the predictions
utils.save_predictions_to_csv(predictions_rb, "results/fesf/predictions_fesf_rb.csv", target_names= ["Robot Protective Stop"])
utils.save_predictions_to_csv(predictions_gl, "results/fesf/predictions_fesf_gl.csv", target_names= ["Grip Lost"])
utils.save_predictions_to_csv(predictions_tc, "results/fesf/predictions_fesf_tc.csv", target_names= ["Tool Current"])

Predictions saved at results/fesf/predictions_fesf_rb.csv
Predictions saved at results/fesf/predictions_fesf_gl.csv
Predictions saved at results/fesf/predictions_fesf_tc.csv


In [22]:
# Model Evaluation results of the 1st Model "Robot Protective Stop" [Classification]
metrics_rb = utils.calculate_metrics(tel_rb, predictions_rb, "classification")
utils.display_metrics(metrics_rb, "classification")
utils.save_metrics_to_csv(metrics_rb, 'results/fesf/metrics_fesf_rb.csv', "classification")

+-----------+---------+
| Metric    |   Value |
+===========+=========+
| Accuracy  |  0.9448 |
+-----------+---------+
| Precision |  0.9227 |
+-----------+---------+
| Recall    |  0.9448 |
+-----------+---------+
| F1-Score  |  0.9336 |
+-----------+---------+
Metrics saved at results/fesf/metrics_fesf_rb.csv


In [23]:
# Model Evaluation results of the 2nd Model "Grip Lost" [Classification]
metrics_gl = utils.calculate_metrics(tel_gl, predictions_gl, "classification")
utils.display_metrics(metrics_gl, "classification")
utils.save_metrics_to_csv(metrics_gl, 'results/fesf/metrics_fesf_gl.csv', "classification")

+-----------+---------+
| Metric    |   Value |
+===========+=========+
| Accuracy  |  0.9597 |
+-----------+---------+
| Precision |  0.9676 |
+-----------+---------+
| Recall    |  0.9597 |
+-----------+---------+
| F1-Score  |  0.9637 |
+-----------+---------+
Metrics saved at results/fesf/metrics_fesf_gl.csv


In [24]:
# Model Evaluation results of the 3rd Model "Tool Current" [Regression]
metrics_tc = utils.calculate_metrics(tel_tc, predictions_tc, "regression")
utils.display_metrics(metrics_tc, "regression")
utils.save_metrics_to_csv(metrics_tc, 'results/fesf/metrics_fesf_tc.csv', "regression")

+----------+---------+
| Metric   |   Value |
+==========+=========+
| MSE      |  0.0085 |
+----------+---------+
| RMSE     |  0.0923 |
+----------+---------+
| MAE      |  0.0523 |
+----------+---------+
| MAPE     |  0.397  |
+----------+---------+
Metrics saved at results/fesf/metrics_fesf_tc.csv


In [25]:
# Freeing up memory
gc.collect()

1049